참고: https://velog.io/@jaehyeong/CNN-%EB%AA%A8%EB%8D%B8%EC%9D%84-%ED%86%B5%ED%95%9C-%EC%9E%90%EB%8F%99%EC%B0%A8-%EC%82%AC%EA%B3%A0-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EB%B6%84%EB%A5%98

# 합성곱 신경망(Convolutional Neural Network, CNN) 모델
- 모델은 이미지 분류의 정석으로 불리는 CNN(Convolution Neural Network) 모델을 활용하였습니다.<br>
- 총 3개의 층으로 구성하였고, 활성화 함수로는 relu 및 softmax 함수를 적용하였습니다.
- dropout도 적용하여 과적합을 방지하였습니다.

## 데이터 불러오기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3 # 검증된 학습
from tensorflow.python.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
## 카테고리 지정하기
categories = ['Plastic', 'RecycledPlastic', 'Can', 'Glass', 'Paper', 'trash']
nb_classes = len(categories)

In [3]:
## 이미지 크기 지정하기
image_w = 64
image_h = 64

In [4]:
## 데이터 열기 
X_train, X_test, y_train, y_test = np.load("./image/6obj.npy", allow_pickle=True)

In [5]:
## train, test set 확인
print('train 이미지:', X_train.shape)
print('test 이미지:', X_test.shape)
print('train label:', y_train.shape)
print('test label:', y_test.shape)

train 이미지: (42448, 64, 64, 3)
test 이미지: (10613, 64, 64, 3)
train label: (42448, 6)
test label: (10613, 6)


## 데이터 정규화

In [6]:
## 데이터 정규화하기(0 ~ 1사이로)
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float") / 255

## 모델 생성(합성곱 신경망, CNN)

In [7]:
## 모델 구조 정의
model = Sequential()
# 얼리스탑핑
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

## 1층
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))  # 합성곱층 1 (64, 64, 3)
model.add(Activation('relu'))                                                 # 활성화 함수
model.add(MaxPooling2D(pool_size=(2, 2)))                                     # 풀링층 1
model.add(Dropout(0.2))       # 과적합 방지

## 2층
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

## 3층
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

## 4층
model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))                          # 풀링층 
model.add(Activation('relu'))
model.add(Dropout(0.2))

## 완전연결층
model.add(Flatten())   # 1차원 벡터 형태로 reshape
model.add(Dense(512))  # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

## 출력층
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

## 모델 확인
model.summary()

## 모델 compile
model.compile(loss='categorical_crossentropy',  # 최적화 함수 지정
              optimizer='Adam',                 # RMSProp
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0         
                                                        

### 전이학습모델생성

In [ ]:
# base_model =  ResNet50(weights='imagenet', include_top=False, input_shape=(64,64,3))
# base_model.summary()
# model = Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(6, activation='softmax'))
# #모델확인
# model.summary()

## 모델 학습

In [8]:
%%time
## 학습 완료한 모델 저장 경로
hdf5_file = "C:/Users/ADMIN/Documents/수업파일/221214(Semi)/save Us(Earth)/결과/23/6obj-model.hdf5"

## 기존에 학습된 모델 불러들이기
if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
    
## 학습한 모델이 없으면 파일로 저장
else:
    history = model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_test, y_test), callbacks=[early_stopping_callback])
    model.save_weights(hdf5_file)

CPU times: total: 0 ns
Wall time: 19.2 ms


## 모델의 오차와 정확도

In [9]:
## 모델 평가하기 
score = model.evaluate(X_test, y_test)

print('loss=', score[0])      # loss
print('accuracy=', score[1])  # accuracy

332/332 [==============================] - 4s 12ms/step - loss: 0.3430 - accuracy: 0.8842
loss= 0.3430121839046478
accuracy= 0.8841986060142517


## 그래프

In [ ]:
import pandas as pd

df_history = pd.DataFrame(history.history)
df_history.plot()

## 이미지 예측

In [10]:
from PIL import Image

## 예측할 이미지 
test_image = './image/test_img/test3.jpg'

## 이미지 전처리
img = Image.open(test_image)       # 이미지 열기
img = img.convert('RGB')           # RGB로 변환
img = img.resize((64, 64))         # 이미지 resize
data = np.asarray(img)             # 이미지 -> numpy 배열 변환
data = data.astype('float') / 255  # 데이터 정규화
data = data.reshape(-1, 64, 64, 3)

## 예측
pred = model.predict(data)  
result = [np.argmax(value) for value in pred]  # 예측 값 중 가장 높은 클래스 반환
print('이것은', categories[result[0]], '입니다.')

이것은 Plastic 입니다.


## 이미지 예측(폴더 전체)

In [15]:
test_dir = 'C:/Users/ADMIN/Documents/수업파일/221214(Semi)/save Us(Earth)/image/test_img/'
fnames = os.listdir(test_dir)

In [16]:
from PIL import Image

for i in fnames:
    ## 예측할 이미지
    test_image = test_dir + i
    
    ## 이미지 전처리
    img = Image.open(test_image)       # 이미지 열기
    img = img.convert('RGB')           # RGB로 변환
    img = img.resize((64, 64))         # 이미지 resize
    data = np.asarray(img)             # 이미지 -> numpy 배열 변환
    data = data.astype('float') / 255  # 데이터 정규화
    data = data.reshape(-1, 64, 64, 3)
    
    ## 예측
    pred = model.predict(data)  
    result = [np.argmax(value) for value in pred]  # 예측 값 중 가장 높은 클래스 반환
    print(i, '사진은', categories[result[0]], '입니다.')

bottle.jfif 사진은 Glass 입니다.
bottle2.jfif 사진은 Plastic 입니다.
bottle3.jfif 사진은 Plastic 입니다.
CAN (1).jpg 사진은 Can 입니다.
CAN (10).jpg 사진은 Can 입니다.
CAN (2).jpg 사진은 RecycledPlastic 입니다.
CAN (3).jpg 사진은 Plastic 입니다.
CAN (4).jpg 사진은 Can 입니다.
CAN (5).jpg 사진은 Can 입니다.
CAN (6).jpg 사진은 Can 입니다.
CAN (7).jpg 사진은 Plastic 입니다.
CAN (8).jpg 사진은 Can 입니다.
CAN (9).jpg 사진은 Paper 입니다.
NoLabel (1).jpg 사진은 RecycledPlastic 입니다.
NoLabel (10).jpg 사진은 RecycledPlastic 입니다.
NoLabel (2).jpg 사진은 RecycledPlastic 입니다.
NoLabel (3).jpg 사진은 RecycledPlastic 입니다.
NoLabel (4).jpg 사진은 RecycledPlastic 입니다.
NoLabel (5).jpg 사진은 RecycledPlastic 입니다.
NoLabel (6).jpg 사진은 RecycledPlastic 입니다.
NoLabel (7).jpg 사진은 RecycledPlastic 입니다.
NoLabel (8).jpg 사진은 RecycledPlastic 입니다.
NoLabel (9).jpg 사진은 RecycledPlastic 입니다.
paper1.jfif 사진은 RecycledPlastic 입니다.
paper2.jfif 사진은 RecycledPlastic 입니다.
PET (1).jpg 사진은 Plastic 입니다.
PET (10).jpg 사진은 Plastic 입니다.
PET (2).jpg 사진은 Plastic 입니다.
PET (3).jpg 사진은 Plastic 입니다.
PET (4).jpg 사진은 Plastic 입니다.
PET (5).j